# Import the libs needed to make everything work

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Pull data from files and use pands tro read json

In [3]:
data = pd.read_json('/content/merged_data.json')

# Seprate X and y using pandas

In [4]:
data

,004Gt3gcsotuiYmz,00FTGIZEd6B8zQ4U,00HGGqBRf1kzPRlg,00IzI5ynahBVtC9l,00JPgT4b72LdNFwS,00KGyPQl5L2NTblB,00ZME4ZkHAEIC7dP,00eP4XD3VdMmHITE,00nSJCUKXfl3rUnl,00nTvY9J61A50DLK,...,zzMV2qp1O6sVNvnj,zzPIdCpN1j4NNMT2,zzZMTdCEiAiVRRO3,zzczgqT7mQHxQYIx,zzhDio9IIUMiXrBk,zzjFSqASfeSPDsh1,zzkjinLBHepHRGbe,zzmg5VFM53K2PfDX,zzw8ORfncyrlNR6J,zzwPV6NCYsMiDb0a
ID,004Gt3gcsotuiYmz,00FTGIZEd6B8zQ4U,00HGGqBRf1kzPRlg,00IzI5ynahBVtC9l,00JPgT4b72LdNFwS,00KGyPQl5L2NTblB,00ZME4ZkHAEIC7dP,00eP4XD3VdMmHITE,00nSJCUKXfl3rUnl,00nTvY9J61A50DLK,...,zzMV2qp1O6sVNvnj,zzPIdCpN1j4NNMT2,zzZMTdCEiAiVRRO3,zzczgqT7mQHxQYIx,zzhDio9IIUMiXrBk,zzjFSqASfeSPDsh1,zzkjinLBHepHRGbe,zzmg5VFM53K2PfDX,zzw8ORfncyrlNR6J,zzwPV6NCYsMiDb0a
authors,N. R. Kleinfield,Anya Kamenetz,Ian Mason,Analysis Stephen Collinson,Sen. Rand Paul,,"Jim Vandehei, Mike Allen","Emily Stewart, Terry Nguyen, Rebecca Jennings,...","Jim Meyers, John Bachman",,...,Michelle Andrews,,Eleanor Beardsley,"Michael Tomasky, Special Correspondent",Andrew Sullivan,Ali Elkin,Casey Newton,Andrea Riquier,Jason Devaney,Julia Munslow
bias,0,0,2,0,2,0,0,0,2,1,...,1,2,1,0,0,1,0,2,2,1
bias_text,left,left,right,left,right,left,left,left,right,center,...,center,right,center,left,left,center,left,right,right,center
content,"Besides his most recent trip to Quetta , Mr. R...",Poll : Prestigious Colleges Wo n't Make You Ha...,"House Speaker Paul Ryan , at a private dinner ...",( CNN ) President Donald Trump has reason to h...,The controversial immigration-reform bill that...,Dick Morris is working with Republican Nationa...,Tired of presidential politics ? Get over it :...,Is Maine Republican Sen. Susan Collins being b...,Wall Street Journal economics expert Stephen M...,The left believes Sanders ’ s chances have imp...,...,Is It Time To Make Medical And Family Leave Pa...,Democrat Terry McAuliffe won the Virginia gove...,When terrorists attacked a satirical magazine ...,So the familiar pattern asserts itself again ....,"Like any religion , wokeness understands the n...",Ahead of President Barack Obama 's visit to th...,"Recently , a friend told me he wanted to spend...","As the coronavirus outbreak claims lives , job...",China promised it would not restrict exports o...,“ The 360 ” shows you diverse perspectives on ...
content_original,"Besides his most recent trip to Quetta, Mr. Ra...",Poll: Prestigious Colleges Won't Make You Happ...,"House Speaker Paul Ryan, at a private dinner e...",(CNN) President Donald Trump has reason to hop...,ANALYSIS/OPINION:\n\nThe controversial immigra...,Dick Morris is working with Republican Nationa...,"AP Photos Republicans, 2016: In full swing\n\n...",Is Maine Republican Sen. Susan Collins being b...,Urgent Poll:\n\nWho Won the Presidential Debat...,From the Left\n\nThe left believes Sanders’s c...,...,Is It Time To Make Medical And Family Leave Pa...,Democrat Terry McAuliffe won the Virginia gove...,The French Debate: Free Speech Versus Hate Spe...,So the familiar pattern asserts itself again. ...,Photo: Mesamong/Getty Images\n\nLike any relig...,SHARE THIS ARTICLE Share Tweet Post Email\n\nP...,"Recently, a friend told me he wanted to spend ...","As the coronavirus outbreak claims lives, jobs...",China promised it would not restrict exports o...,“The 360” shows you diverse perspectives on th...
date,2016-09-20,2014-05-06,2017-09-12,2019-07-11,2013-05-24,2013-03-28,2012-11-21,2018-09-12,2012-10-09,,...,2015-04-22,2013-11-06,2015-02-10,2016-08-27,2019-09-20,2015-01-12,2019-11-06,2020-04-16,2020-04-05,2020-06-18
source,New York Times - News,Ezra Klein,Breitbart News,Guest Writer - Left,Washington Times,Media Matters,Politico,Vox,Newsmax,The Flip Side,...,NPR Online News,Fox News,NPR Online News,Daily Beast,New York Magazine,Bloomberg,The Verge,MarketWatch,Newsmax - News,Yahoo! The 360
source_url,www.nytimes.com,www.npr.org,www.breitbart.com,www.cnn.com,www.washingtontimes.com,www.mediamatters.org,www.politico.com,www.vox.com,www.newsmax.com,www.theflipside.io,...,www.npr.org,www.foxnews.com,www.npr

In [5]:
X = data.iloc[4].astype(str)
y = data.iloc[2].astype(int)

# Split the data

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# use textvectortization to semi prep the text

In [7]:
text_vec = tf.keras.layers.TextVectorization(max_tokens=10000, output_mode='int')
text_vec.adapt(X_train)

In [8]:
sample_text = np.array(["This is a sample text."])
print(text_vec(sample_text))

tf.Tensor([[  34   12    6 5324 2231]], shape=(1, 5), dtype=int64)


# Build the model

In [9]:
X_train

xKF0wr490dScf5cT    The jobs recovery is chugging along , but the ...
Z55DRzCgkMjs1Uck    University of Minnesota Law School professor F...
hj8lcnqZTfjGV7fk    Speaking at the World Economic Forum in Davos ...
ciYELNfS2ydOx2dL    I was in Wisconsin for the labor uprising of 2...
ObBWe7KRg4jUHjD4    CLOSE Trump Nation is more than a demographic ...
                                          ...                        
RZGPUmGt6zeMkKYt    On Tuesday , President Obama delivered his ann...
AKohcWlhYtyFQjgv    President Donald Trump on Thursday called for ...
IbBrsKW4MiO7T1LG    I watch election results to gauge whether Amer...
1PaVAnNTndDIwx8Z    House Minority Leader Nancy Pelosi gave more d...
PsBRQdaNyuNh6D0P    Trump ’ s push to expand oil and gas drilling ...
Name: content, Length: 30043, dtype: object

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    text_vec,
    tf.keras.layers.Embedding(input_dim=len(text_vec.get_vocabulary()),
                              output_dim=64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
1878/1878 [==============================] - 552s 289ms/step - loss: 0.9022 - accuracy: 0.5636 - val_loss: 0.8222 - val_accuracy: 0.6167
Epoch 2/10
1878/1878 [==============================] - ETA: 0s - loss: 0.7252 - accuracy: 0.6855